In [ ]:
import numpy as np
import pandas as pd
import asyncio
import aiohttp
import json
import csv

In [16]:
import requests
import json
from datetime import datetime
import numpy as np
import pandas as pd

def get_markets(api = "cryptowatch"):
    """get markets available in API"""
    if api == "cryptowatch":
        try:
            r = requests.get("https://api.cryptowat.ch/markets").json()['result']
            exchanges = {item['exchange'] for item in r}
        except:
            print("Error: disfunctional API from Cryptowatch")
    return exchanges

def get_pairs(api = "cryptowatch"):
    """get pairs available in API"""
    if api == "cryptowatch":
        try:
            r = requests.get("https://api.cryptowat.ch/markets").json()['result']
            pairs = {item['pair'] for item in r}
        except:
            print("Error: disfunctional API from Cryptowatch")
    return pairs



In [20]:
def get_ohlc(market, pair, period = 24, api = "cryptowatch", before = "", after = ""):
    """get OHLC prices available in API"""
    period *= 3600
    period = str(period)
    params = []
    url = "https://api.cryptowat.ch/markets/" + market + "/" + pair + "/ohlc"
    print(url)
    if (not before or not after):
        params = [('before', before), ('after', after)]
    if api == "cryptowatch":
        try:
            r = requests.get(url).json()['result'][period]
            #ohlc = [item['pair'] for item in r]
            return pd.DataFrame(r, columns = ['period','open','high','low','close','volume','neg'])
        except:
            print("Error: disfunctional API from Cryptowatch")
            
#columns = ['period','open','high','low','close','volume','neg']    

In [21]:
a = get_ohlc("bitfinex", "btcusd")

https://api.cryptowat.ch/markets/bitfinex/btcusd/ohlc


In [23]:
a.tail()

,period,open,high,low,close,volume,neg
495,1531440000,6377.7,6377.7,6079.9,6249.0,23411.469,145159060.0
496,1531526400,6248.7,6339.9,6125.0,6220.0,18539.768,115527330.0
497,1531612800,6219.7,6320.0,6179.2,6250.0,8081.261,50436940.0
498,1531699200,6250.0,6389.0,6230.3,6347.2,10118.092,64132416.0
499,1531785600,6347.2,6671.6,6327.1,6596.5,19173.992,124880856.0


In [10]:
np.random.random(5)

array([0.25554614, 0.46100205, 0.85671336, 0.49220139, 0.02059016])